In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

### 2.11

In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat',
                      sep='\n',
                      delimiter='::',
                      header=None,
                      names=['UserID', 'MovieID','Rating','Timestamp'],
                      engine='python')


In [5]:
movies = pd.read_csv('ml-1m/movies.dat',sep='\n',
                     delimiter='::',
                     header=None, 
                     names=['MovieID', 'Title','Genres'], 
                     engine='python')

In [6]:
users = pd.read_csv('ml-1m/users.dat',sep='\n',
                     delimiter='::',
                     header=None, 
                     names=['UserID', 'Gender','Age', 'Occupation','Zip-code'], 
                     engine='python')

In [7]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
movies.set_index("MovieID")
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movies[["Title", "Year"]] = movies.Title.str.extract(r"(.*) \((\d{4})\)")

movies.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995


In [12]:
movies['MovieID'].count()

3883

In [13]:
movies['Year']  = movies['Year'].astype(int)

In [14]:
countByYear = movies.groupby('Year').count()
countByYear.head()

,MovieID,Title,Genres
Year,,,
1919,3,3,3
1920,2,2,2
1921,1,1,1
1922,2,2,2
1923,3,3,3


In [15]:
countByYear = movies.Year.value_counts(sort=True)
countByYear.head()

1996    345
1995    342
1998    337
1997    315
1999    283
Name: Year, dtype: int64

In [16]:
sexRatio = pd.crosstab(users.Gender, users.Age)
sexRatio

Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,78,298,558,338,189,146,102
M,144,805,1538,855,361,350,278


In [17]:
allGenresAccurances = pd.Series(np.concatenate(movies.Genres.str.split('|'), axis=0))
allGenresAccurances.columns = ['Genre']
allGenresAccurances.value_counts(sort=True)

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Western          68
Fantasy          68
Film-Noir        44
dtype: int64

In [20]:
ratings_by_movie = pd.merge(movies,ratings, left_index = True, right_on="MovieID").groupby('Title').mean()['Rating'].sort_values(ascending=False)
ratings_by_movie = ratings_by_movie.reset_index()
ratings_by_movie.head()

,Title,Rating
0,"Perils of Pauline, The",5.0
1,Beautiful People,5.0
2,"Cup, The (Ph�rpa)",5.0
3,American Pop,5.0
4,Passion in the Desert,5.0


In [22]:
ratings_count = pd.merge(movies,ratings, left_index=True,right_on='MovieID').groupby('Title').count()['Rating'] 
movies_with_100_ratings = pd.Series(ratings_count[ratings_count > 100]).reset_index()

pd.merge(movies_with_100_ratings,ratings_by_movie, on='Title').sort_values('Rating_y', ascending=False).head()

,Title,Rating_x,Rating_y
1369,Popeye,628,4.560510
1692,Strawberry and Chocolate (Fresa y chocolate),2227,4.554558
941,Kansas City,2223,4.524966
959,Kim,657,4.520548
746,Guardian Angel,1783,4.517106


In [36]:
user_ratings = pd.merge(users.set_index('UserID'),ratings, left_index=True, right_on='UserID').set_index('MovieID')
user_ratings.head()

,Gender,Age,Occupation,Zip-code,UserID,Rating,Timestamp
MovieID,,,,,,,
1193,F,1,10,48067,1,5,978300760
661,F,1,10,48067,1,3,978302109
914,F,1,10,48067,1,3,978301968
3408,F,1,10,48067,1,4,978300275
2355,F,1,10,48067,1,5,978824291


In [62]:
user_ratings_movies = pd.merge(user_ratings, movies, left_index=True, right_on='MovieID')
best_movies_by_gender = user_ratings_movies.groupby(['Gender', 'Title']).agg(['mean', 'count'])['Rating']
best_movies_by_gender = best_movies_by_gender[best_movies_by_gender['count'] > 100]
best_movies_by_gender.reset_index().sort_values(by=['mean','Gender'], ascending=False).drop_duplicates(['Gender'])


,Gender,Title,mean,count
162,F,"Close Shave, A",4.644444,180
1450,M,"Godfather, The",4.583333,1740


In [76]:
mean_year_by_age = user_ratings_movies.groupby(['Age']).agg(['mean'])['Year']
np.round(mean_movies_by_age)

,mean
Age,
1,1989.0
18,1990.0
25,1988.0
35,1985.0
45,1984.0
50,1982.0
56,1982.0


In [89]:
user_ratings_movies_female = user_ratings_movies[user_ratings_movies['Gender'] == 'F']
user_ratings_movies_male = user_ratings_movies[user_ratings_movies['Gender'] == 'M']

print(pd.Series(user_ratings_movies_female.Genres.str.split(pat = '|', expand=True).values.ravel()).describe().iloc[2])
print(pd.Series(user_ratings_movies_male.Genres.str.split(pat = '|', expand=True).values.ravel()).describe().iloc[2])

Drama
Comedy


# 2.10

In [5]:
import os

In [6]:
import sqlite3

In [9]:
flights = pd.read_csv('nycflights13_flights.csv.gz',
                      comment ='#',
                      engine='python')


In [10]:
planes = pd.read_csv('nycflights13_planes.csv.gz',
                      comment ='#',
                      engine='python')


In [11]:
weather = pd.read_csv('nycflights13_weather.csv.gz',
                      comment ='#',
                      engine='python')

In [12]:
airports = pd.read_csv('nycflights13_airports.csv.gz',
                      comment ='#',
                      engine='python')

In [13]:
airlines = pd.read_csv('nycflights13_airlines.csv.gz',
                      comment ='#',
                      engine='python')

In [14]:
db_name = 'nycflights13.db'
os.remove(os.path.join(os.getcwd(), db_name))

In [15]:
db_name = 'nycflights13.db'
con = sqlite3.connect(os.path.join(os.getcwd(), db_name))
planes.to_sql('planes', con);
flights.to_sql('flights', con);
airlines.to_sql('airlines', con);
weather.to_sql('weather', con);
airports.to_sql('airports', con);

In [46]:
sql_result3 = pd.read_sql_query('Select count(*), engine from planes group by engine', con)
sql_result3

,count(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


In [47]:
frame3 = pd.DataFrame(planes.groupby('engine').count()['manufacturer']).reset_index()[['manufacturer', 'engine']]
frame3.columns = ['count(*)', 'engine']
frame3


,count(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


In [50]:
sql_result4 = pd.read_sql_query('select count(*),engine, type from planes group by engine,type', con)
sql_result4

,count(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


In [79]:
frame4 = pd.DataFrame(planes.groupby(['engine', 'type']).count()['manufacturer']).reset_index()
frame4.columns = ['engine','type','count(*)']
frame4 = frame4[['count(*)', 'engine', 'type']]
frame4

,count(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


In [91]:
frame4b = pd.crosstab(planes.engine, planes.type).unstack().reset_index()
frame4b = frame4b[frame4b[0] != 0]
frame4b.columns = ['type', 'engine', 'count(*)']
frame4b.reset_index()[['count(*)','engine', 'type']]

,count(*),engine,type
0,5,Reciprocating,Fixed wing multi engine
1,2750,Turbo-fan,Fixed wing multi engine
2,535,Turbo-jet,Fixed wing multi engine
3,2,Turbo-prop,Fixed wing multi engine
4,2,4 Cycle,Fixed wing single engine
5,23,Reciprocating,Fixed wing single engine
6,5,Turbo-shaft,Rotorcraft


In [77]:
sql_result5 = pd.read_sql_query('select min(year), avg(year), max(year),engine,manufacturer from planes group by engine, manufacturer',con)
sql_result5

,min(year),avg(year),max(year),engine,manufacturer
0,1975.0,1975.000000,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.000000,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L
5,1959.0,1971.142857,1983.0,Reciprocating,CESSNA
6,2007.0,2007.000000,2007.0,Reciprocating,CIRRUS DESIGN CORP
7,1959.0,1959.000000,1959.0,Reciprocating,DEHAVILLAND
8,1956.0,1956.000000,1956.0,Reciprocating,DOUGLAS
9,2007.0,2007.000000,2007.0,Reciprocating,FRIEDEMANN JON


In [101]:
frame5 = pd.DataFrame(planes.groupby(['engine', 'manufacturer']).agg(['min','mean','max'])['year']).reset_index()
frame5.columns = ['engine','manufacturer','min(year)','avg(year)','max(year)']
frame5[['min(year)','avg(year)','max(year)','engine','manufacturer']]

,min(year),avg(year),max(year),engine,manufacturer
0,1975.0,1975.000000,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.000000,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L
5,1959.0,1971.142857,1983.0,Reciprocating,CESSNA
6,2007.0,2007.000000,2007.0,Reciprocating,CIRRUS DESIGN CORP
7,1959.0,1959.000000,1959.0,Reciprocating,DEHAVILLAND
8,1956.0,1956.000000,1956.0,Reciprocating,DOUGLAS
9,2007.0,2007.000000,2007.0,Reciprocating,FRIEDEMANN JON


In [102]:
sql_result6 = pd.read_sql_query('select * from planes where speed is not null', con)
sql_result6

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
5,1037,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
6,1190,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
7,1430,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
8,1480,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
9,1515,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


In [112]:
frame6 = planes[~planes.speed.isnull()]
frame6

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
1037,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
1190,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
1430,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
1480,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
1515,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


In [110]:
sql_result7 = pd.read_sql_query('select tailnum from planes where year >= 2010', con)
sql_result7

,tailnum
0,N127UW
1,N128UW
2,N150UW
3,N151UW
4,N152UW
5,N153UW
6,N154UW
7,N155UW
8,N156UW
9,N157UW


In [135]:
frame7 = pd.DataFrame(planes[['year','tailnum']][planes.year >= 2010]).reset_index(drop=True)
pd.DataFrame(frame7['tailnum']).equals(sql_result7)

True

In [139]:
sql_result8 = pd.read_sql_query('select tailnum from planes where seats between 100 and 200 limit 20', con)
sql_result8

,tailnum
0,N102UW
1,N103US
2,N104UW
3,N105UW
4,N107US
5,N108UW
6,N109UW
7,N110UW
8,N111US
9,N11206


In [153]:
frame8 = pd.DataFrame(planes[['tailnum','seats']][planes.seats >= 100][planes.seats <= 200]).reset_index(drop=True)['tailnum']
frame8.head(20)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0     N102UW
1     N103US
2     N104UW
3     N105UW
4     N107US
5     N108UW
6     N109UW
7     N110UW
8     N111US
9     N11206
10    N112US
11    N113UW
12    N114UW
13    N117UW
14    N118US
15    N119US
16    N12109
17    N12114
18    N12116
19    N12125
Name: tailnum, dtype: object

In [155]:
sql_result9 = pd.read_sql_query('select * from planes where manufacturer in ("BOEING","AIRBUS","EMBRAER")',con)
sql_result9

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
1,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,None,Turbo-fan
2,10,N11106,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
3,11,N11107,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
4,12,N11109,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
5,13,N11113,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
6,14,N11119,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
7,15,N11121,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
8,16,N11127,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
9,17,N11137,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan


In [161]:
frame9 = pd.DataFrame(planes[planes.manufacturer.isin(["BOEING","AIRBUS","EMBRAER"])]).reset_index()
frame9

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan
2,10,N11106,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
3,11,N11107,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
4,12,N11109,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
5,13,N11113,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
6,14,N11119,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
7,15,N11121,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
8,16,N11127,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
9,17,N11137,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan


In [163]:
sql_result10 = pd.read_sql_query('select * from planes where manufacturer in ("BOEING","AIRBUS","EMBRAER") and seats>300',con)
sql_result10

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,49,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
1,50,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
2,69,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
3,117,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
4,122,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
5,183,N143DA,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
6,241,N1602,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
7,242,N1603,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
8,243,N1604R,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
9,244,N1605,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan


In [166]:
frame10 = pd.DataFrame(planes[planes.manufacturer.isin(["BOEING","AIRBUS","EMBRAER"]) & (planes.seats > 300)]).reset_index()
frame10

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,49,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
1,50,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
2,69,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
3,117,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
4,122,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
5,183,N143DA,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
6,241,N1602,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
7,242,N1603,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
8,243,N1604R,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
9,244,N1605,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan


In [169]:
sql_result11 = pd.read_sql_query('select manufacturer, COUNT(*) from planes where seats>200 group by manufacturer',con)
sql_result11

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [202]:
frame11 = pd.DataFrame(planes[planes.seats > 200]).groupby('manufacturer').count()['tailnum'].reset_index()
frame11.columns = ['manufacturer','COUNT(*)']
frame11

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [203]:
sql_result12 = pd.read_sql_query('select manufacturer, COUNT(*) from planes where seats>200 group by manufacturer having count(*) > 10',con)
sql_result12

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


In [205]:
frame12 = pd.DataFrame(planes[planes.seats > 200]).manufacturer.value_counts().reset_index()
frame12[frame12.manufacturer > 10]

,index,manufacturer
0,BOEING,225
1,AIRBUS,66


In [207]:
sql_result14 = pd.read_sql_query('SELECT manufacturer, COUNT(*) AS howmany FROM planes GROUP BY manufacturer ORDER BY howmany',con)
sql_result14

,manufacturer,howmany
0,AGUSTA SPA,1
1,AVIAT AIRCRAFT INC,1
2,AVIONS MARCEL DASSAULT,1
3,BARKER JACK L,1
4,CANADAIR LTD,1
5,CIRRUS DESIGN CORP,1
6,DEHAVILLAND,1
7,DOUGLAS,1
8,FRIEDEMANN JON,1
9,HURLEY JAMES LARRY,1


In [228]:
planes.manufacturer.value_counts(ascending=True).reset_index().sort_values(by=['manufacturer','index'])

,index,manufacturer
15,AGUSTA SPA,1
11,AVIAT AIRCRAFT INC,1
6,AVIONS MARCEL DASSAULT,1
1,BARKER JACK L,1
9,CANADAIR LTD,1
17,CIRRUS DESIGN CORP,1
10,DEHAVILLAND,1
3,DOUGLAS,1
7,FRIEDEMANN JON,1
14,HURLEY JAMES LARRY,1


In [229]:
sql_result15 = pd.read_sql_query('SELECT manufacturer, COUNT(*) AS howmany FROM planes GROUP BY manufacturer ORDER BY howmany DESC LIMIT 10',con)
sql_result15

,manufacturer,howmany
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14
8,CANADAIR,9
9,CESSNA,9


In [234]:
planes.manufacturer.value_counts(ascending=True).reset_index().sort_values(by=['manufacturer','index'],ascending=False).head(10)

,index,manufacturer
34,BOEING,1630
33,AIRBUS INDUSTRIE,400
32,BOMBARDIER INC,368
31,AIRBUS,336
30,EMBRAER,299
29,MCDONNELL DOUGLAS,120
28,MCDONNELL DOUGLAS AIRCRAFT CO,103
27,MCDONNELL DOUGLAS CORPORATION,14
25,CESSNA,9
26,CANADAIR,9


In [236]:
sql_result16 = pd.read_sql_query('SELECT * FROM planes WHERE year >= 2012 ORDER BY year,seats',con)
sql_result16

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,None,Turbo-shaft
1,815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
2,825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
3,837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
4,2630,N8301J,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
5,2631,N8302F,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
6,2633,N8305E,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
7,2634,N8306H,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
8,2635,N8307K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
9,2636,N8308K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan


In [239]:
planes[planes.year >= 2012].sort_values(by=['year','seats'])

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,NaN,Turbo-shaft
815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
2630,N8301J,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
2631,N8302F,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
2633,N8305E,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
2634,N8306H,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
2635,N8307K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
2636,N8308K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan


In [241]:
sql_result17 = pd.read_sql_query('SELECT * FROM planes WHERE year >= 2012 ORDER BY seats, year',con)
sql_result17

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,None,Turbo-shaft
1,815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
2,825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
3,837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
4,843,N354JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
5,848,N355JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
6,859,N358JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
7,908,N368JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
8,956,N373JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
9,989,N374JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan


In [243]:
planes[planes.year >= 2012].sort_values(by=['seats','year'])

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,NaN,Turbo-shaft
815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
843,N354JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
848,N355JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
859,N358JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
908,N368JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
956,N373JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
989,N374JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan


In [246]:
airports.sample(100)

,faa,name,lat,lon,alt,tz,dst,tzone
795,LIT,Adams Fld,34.729444,-92.224306,262,-6,A,America/Chicago
853,MCW,Mason City Municipal,43.224700,-93.406700,1243,-6,A,America/Chicago
353,DAW,Skyhaven Airport,43.284056,-70.929278,321,-5,A,America/New_York
1195,SBS,Steamboat Springs Airport-Bob Adams Field,40.516250,-106.866306,6882,-7,A,America/Denver
883,MKG,Muskegon County Airport,43.169500,-86.238200,628,-5,A,America/New_York
314,COD,Yellowstone Rgnl,44.520194,-109.023806,5102,-7,A,America/Denver
266,CDI,Cambridge Municipal Airport,39.975028,-81.577583,799,-5,U,America/New_York
1207,SDY,Sidney-Richland Municipal Airport,47.706944,-104.192500,1984,-7,A,America/Denver
253,C47,Portage Municipal Airport,43.560314,-89.482861,825,-6,U,America/Chicago
867,MGM,Montgomery Regional Airport,32.300639,-86.393972,221,-6,A,America/Chicago


In [252]:
airports.sample(frac=0.05)

,faa,name,lat,lon,alt,tz,dst,tzone
370,DKX,Knoxville Downtown Island Airport,35.963833,-83.873667,833,-5,A,America/New_York
69,70J,Cairo-Grady County Airport,30.887977,-84.154735,265,-5,A,America/New_York
1446,ZRP,Newark Penn Station,40.734722,-74.164167,0,-5,A,America/New_York
1224,SHD,Shenandoah Valley Regional Airport,38.263889,-78.896389,1201,-5,A,America/New_York
295,CIU,Chippewa County International Airport,46.250800,-84.472400,800,-5,A,America/New_York
1436,YKM,Yakima Air Terminal McAllister Field,46.568200,-120.544000,1095,-8,A,America/Los_Angeles
725,KKH,Kongiganak Airport,59.960833,-162.881111,30,-9,A,America/Anchorage
148,ARV,Lakeland,45.927778,-89.730833,1629,-6,A,America/Chicago
1441,ZBP,Penn Station,39.307222,-76.615556,66,-5,A,America/New_York
658,ILN,Wilmington Airborne Airpark,39.427920,-83.792118,1077,-5,U,America/New_York


In [254]:
airports.head(100)

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
5,0A9,Elizabethton Municipal Airport,36.371222,-82.173417,1593,-5,A,America/New_York
6,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
7,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
8,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
9,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles


In [256]:
airports.tail(100)

,faa,name,lat,lon,alt,tz,dst,tzone
1358,UPP,Upolu,20.265256,-155.859989,96,-10,A,Pacific/Honolulu
1359,UST,St. Augustine Airport,29.959167,-81.339722,10,-5,A,America/New_York
1360,UT3,Hite Airport,37.891655,-110.384029,3840,-7,A,America/Denver
1361,UTM,Tunica Municipal Airport,34.681000,-90.346700,194,-6,A,America/Chicago
1362,UTO,Indian Mountain Lrrs,65.992794,-153.704289,1220,-9,A,America/Anchorage
1363,UUK,Ugnu-Kuparuk Airport,70.330833,-149.597500,67,-9,A,America/Anchorage
1364,UUU,Newport State,41.533056,-71.282222,172,-5,A,America/New_York
1365,UVA,Garner Field,29.215429,-99.748962,942,-6,A,America/Chicago
1366,VAD,Moody Afb,30.967833,-83.193000,233,-5,A,America/New_York
1367,VAK,Chevak Airport,61.533800,-165.584000,75,-9,A,America/Anchorage


In [264]:
A = airports.head(10)
B = airports[6:16]

In [280]:
pd.concat([A,B]).drop_duplicates()

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
5,0A9,Elizabethton Municipal Airport,36.371222,-82.173417,1593,-5,A,America/New_York
6,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
7,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
8,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
9,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles


In [281]:
A.append(B, ignore_index = True)

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
5,0A9,Elizabethton Municipal Airport,36.371222,-82.173417,1593,-5,A,America/New_York
6,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
7,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
8,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
9,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles


In [278]:
A.merge(B)

,faa,name,lat,lon,alt,tz,dst,tzone
0,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
1,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
2,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
3,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles


In [290]:
A[~A.isin(B)].dropna()

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044.0,-5.0,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264.0,-6.0,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801.0,-6.0,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523.0,-5.0,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11.0,-5.0,A,America/New_York
5,0A9,Elizabethton Municipal Airport,36.371222,-82.173417,1593.0,-5.0,A,America/New_York
